In [ ]:
"""
Result: 0.79424

Logistic

Missing Value Imputation Strategies:
spending columns → 0
age -> median
categorical → 'Unknown'

New Features:
None

Deleted Features:
Name
"""

"\nResult: 0.79728\n\nLogistic\n\nMissing Value Imputation Strategies:\nspending columns → 0\nage -> median\ncategorical → 'Unknown'\n\nNew Features:\nNone\n\nDeleted Features:\nName\n"

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
import missingno as msno

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

y = train["Transported"].astype(int)

X_train = train.drop(columns=["Transported"])
X_test = test.copy()

In [4]:
X_train['Group'] = X_train['PassengerId'].str.split('_').str.get(0)
X_train['Number'] = X_train['PassengerId'].str.split('_').str.get(1)
X_train['Group_size'] = X_train.groupby('Group')['PassengerId'].transform('count')
X_train['is_alone'] = (X_train['Group_size'] == 1).astype(int)

X_test['Group'] = X_test['PassengerId'].str.split('_').str.get(0)
X_test['Number'] = X_test['PassengerId'].str.split('_').str.get(1)
X_test['Group_size'] = X_test.groupby('Group')['PassengerId'].transform('count')
X_test['is_alone'] = (X_test['Group_size'] == 1).astype(int)

pid_test = test["PassengerId"]
X_train = X_train.drop(columns=['Group','Number','PassengerId'])
X_test = X_test.drop(columns=['Group','Number','PassengerId'])

In [5]:
# spending columns → 0
spend_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
X_train[spend_cols] = X_train[spend_cols].fillna(0)
X_test[spend_cols] = X_test[spend_cols].fillna(0)

# Age → median
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].median())
X_test['Age'] = X_test['Age'].fillna(X_train['Age'].median())

# categorical → 'Unknown'
cat_cols = ['HomePlanet','CryoSleep','Destination','Cabin','VIP']
for c in cat_cols:
    X_train[c] = X_train[c].fillna('Unknown')
    X_test[c] = X_test[c].fillna('Unknown')


In [6]:
drop_cols = ['Name', 'Cabin']
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [8]:
pid_test = test["PassengerId"]       

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# train / test 컬럼 정렬
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

In [9]:

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gb = GradientBoostingClassifier(
    n_estimators=200,     
    learning_rate=0.07,  
    max_depth=3,       
    subsample=0.8,    
    random_state=42
)

gb.fit(X_train, y_train)

val_pred = gb.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_pred))


Validation Accuracy: 0.7883841288096607


In [16]:
test_pred = gb.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": pid_test,
    "Transported": test_pred.astype(bool)
})

submission.to_csv("./submission/v7_gb.csv", index=False)


In [17]:
feat_imp = (
    pd.Series(gb.feature_importances_, index=X_train.columns)
      .sort_values(ascending=False)
)

print(feat_imp.head(15))


CryoSleep_True             0.227336
CryoSleep_False            0.131462
Spa                        0.128156
VRDeck                     0.115270
RoomService                0.104147
FoodCourt                  0.091447
HomePlanet_Earth           0.064801
ShoppingMall               0.048453
Age                        0.042027
HomePlanet_Europa          0.019025
Group_size                 0.006113
Destination_55 Cancri e    0.005826
HomePlanet_Mars            0.005120
Destination_TRAPPIST-1e    0.003450
HomePlanet_Unknown         0.002500
dtype: float64
